<a href="https://colab.research.google.com/github/sheetalmk/projects/blob/main/iimk_context_based_recommender_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=0293e5e63b1d120a2ff132754d5dcb81391f5566b682d1dc58c6255f1ae5bddc
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [4]:
data=pd.read_csv('/content/myntra_products_catalog.csv')

In [5]:
df=data.copy()


In [6]:
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


Prepare data

In [7]:
import spacy
from nltk.tokenize import word_tokenize
sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words
all_stopwords.add('&')
all_stopwords.add(',')
all_stopwords.add('.')
all_stopwords.add('@')
all_stopwords.add('/')
all_stopwords.add(':')
all_stopwords.add('?')

In [8]:
def remove_Stopingwords_Punctuation(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
    return " ".join(tokens_without_sw)

In [12]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
df["ProductName"]=df["ProductName"].apply(lambda x : remove_Stopingwords_Punctuation(x))

In [15]:
df["Description"]=df["Description"].apply(lambda x : remove_Stopingwords_Punctuation(x))
#Create a new column Feature_Set in the df DataFrame by concatenating multiple columns i

In [16]:
df["Feature_Set"]=df["ProductBrand"]+df["ProductName"]+df["Gender"]+df["Description"]+df["PrimaryColor"]

In [17]:
subset=df[["ProductID","Feature_Set"]]

In [18]:
subset

,ProductID,Feature_Set
0,10017413,DKNYDKNY Unisex Black Grey Printed Medium Trol...
1,10016283,EthnoVogueEthnoVogue Women Beige Grey Made Mea...
2,10009781,SPYKARSPYKAR Women Pink Alexa Super Skinny Fit...
3,10015921,RaymondRaymond Men Blue Self-Design Single-Bre...
4,10017833,ParxParx Men Brown Off-White Slim Fit Printed ...
...,...,...
12486,10262843,Pepe JeansPepe Jeans Men Black Hammock Slim Fi...
12487,10261721,MochiMochi Women Gold-Toned Solid HeelsWomenA ...
12488,10261607,612 league612 league Girls Navy Blue White Pri...
12489,10266621,NaN


In [19]:
#Drops any row with missing values from the subset DataFrame.
subset.dropna(axis=0, inplace=True)

<ipython-input-19-50d80a138666>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset.dropna(axis=0, inplace=True)


In [20]:
subset.reset_index(drop=True,inplace=True)

In [21]:
#Loading a pre-trained Sentence Transformer model all-MiniLM-L6-v2.
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [22]:
#Encoding the Feature_Set column of the subset DataFrame using the pre-trained model and stores the embeddings in sentence_embeddings.
sentence_embeddings = model.encode(subset["Feature_Set"])

In [23]:
sentence_embeddings.shape

(11597, 384)

In [24]:
subset["Feature_Set"].values

array(['DKNYDKNY Unisex Black Grey Printed Medium Trolley BagUnisexBlack grey printed medium trolley bag secured TSA lockOne handle trolley retractable handle corner mounted inline skate wheelsOne main zip compartment zip lining compression straps click clasps zip compartment flap zip pocketsWarranty 5 yearsWarranty provided Brand Owner Manufacturer Black',
       'EthnoVogueEthnoVogue Women Beige Grey Made Measure Custom Made Kurta Set JacketWomenBeige Grey measure kurta churidar dupattaBeige measure calf length kurta V-neck three-quarter sleeves lightly padded bust flared hem concealed zip closureGrey solid measure churidar drawstring closureGrey net sequined dupatta printed tapingWhat Made Measure Customised Kurta Set according Bust Length So refer Size Chart pick perfect size.How measure bust Measure arms chest find bust size inchesHow measure Kurta length Measure shoulder till barefoot find kurta length Beige',
       'SPYKARSPYKAR Women Pink Alexa Super Skinny Fit High-Rise Clean

In [25]:
#computing the cosine similarity between the embeddings of the specified
# product_id and all the embeddings in sentence_embeddings.
product_id=10015921
cosine_scores = util.cos_sim(model.encode(subset[subset["ProductID"]==product_id]["Feature_Set"].values[0]), sentence_embeddings)

In [26]:
# Iterating over score five times and selects the product ID with the highest
# cosine similarity score, adds it to recomendations_cos, and marks that score
# as already selected by setting it to -1.
score=cosine_scores[0].tolist()
recomendations_cos=[]
for i in range(0,5):
    maxx=score.index(max(score))
    recomendations_cos.append(subset['ProductID'][maxx])
    score[maxx]=-1

In [27]:
# Printing the list of recommended product IDs.
recomendations_cos

[10015921, 10025539, 10025555, 10025541, 10217093]

In [30]:
#Selecting the rows from the original df DataFrame that have the recommended product IDs and prints them.
df[df["ProductID"].isin(recomendations_cos)]

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,Feature_Set
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,RaymondRaymond Men Blue Self-Design Single-Bre...
1434,10025539,Raymond Men Blue Solid Regular-Fit Single-Brea...,Raymond,Men,6299,5,Blue solid party suitBlue solid regular-fit bl...,Blue,RaymondRaymond Men Blue Solid Regular-Fit Sing...
1822,10025555,Raymond Men Blue Solid Regular-Fit Single-Brea...,Raymond,Men,5849,5,Blue solid party suitBlue solid regular-fit bl...,Blue,RaymondRaymond Men Blue Solid Regular-Fit Sing...
1951,10025541,Raymond Men Blue Solid Regular-Fit Single-Brea...,Raymond,Men,4274,5,Blue solid formal suitBlue solid regular-fit b...,Blue,RaymondRaymond Men Blue Solid Regular-Fit Sing...
9674,10217093,Raymond Men Blue Solid Contemporary Fit Formal...,Raymond,Men,4049,5,Blue solid single-breasted formal suitBlue sol...,Blue,RaymondRaymond Men Blue Solid Contemporary Fit...
